# Using widgets

In [0]:
dbutils.widgets.text("data_source_param", "")
data_source_val = dbutils.widgets.get("data_source_param")

# Run helpers notebooks

In [0]:
%run "../helpers/configuration"

In [0]:
%run "../helpers/functions"

# Ingest and process raw data

In [0]:
# Define schema

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)])

schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                            StructField("driverRef", StringType(), True),
                            StructField("number", IntegerType(), True),
                            StructField("code", StringType(), True),
                            StructField("name", name_schema),
                            StructField("dob", DateType(), True),
                            StructField("nationality", StringType(), True),
                            StructField("url", StringType(), True)
                            ])

In [0]:
# Read json file
drivers_df = spark.read.json(f"{raw_folder_path}/drivers.json", schema = schema)
# display(drivers_df)

In [0]:
# Process data

from pyspark.sql.functions import col, concat, lit

drivers_processed = drivers_df.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("driverRef", "driver_ref") \
    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
    .withColumn("data_source", lit(data_source_val)) \
    .drop(col("url"))

# display(drivers_processed)

In [0]:
# Add ingestion date
drivers_final = add_ingestion_date(drivers_processed)

In [0]:
# Write df to parquet
drivers_final.write.parquet(f"{processed_folder_path}/drivers", mode = "overwrite")

In [0]:
# Check if data has been properly written
display(spark.read.parquet(f"{processed_folder_path}/drivers"))